### GENERATE LINE BASED ON TWO NOUNS
#### required files: 
#### model.back, model_forw, glove embeddings, postag_dict.p, models saved in frost_model and frost_model_forward

In [1]:
import numpy as np
import tensorflow as tf
import argparse
import os
from six.moves import cPickle
import nltk
import time
import queue as Q
from operator import itemgetter
from model_back import Model as Model_back
from model_forw import Model as Model_forw

import itertools
from numpy.random import choice
from collections import defaultdict
from gensim.models.keyedvectors import KeyedVectors
import string
import re
import collections
import sys
import pickle
import getopt
import random
from collections import defaultdict, Counter
import itertools
counts = defaultdict(Counter)
from Traversal import *
from extra_func import *
from Transversal_Glove import Meta_Poetry_Glove

c:\users\hp\anaconda2\envs\translator\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
c:\users\hp\anaconda2\envs\translator\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#LOAD DIRECTORY OF MODELS
text_list = [("data\\frost\\input.txt","save_2"),("data\frost\input.txt","frost_model")]
#np.random.shuffle(text_list)
t = text_list[0][0] #THIS TEXT IS THE VOCAB!
save_dir = text_list[0][1] #THIS IS THE MODEL DIRECTORY
t_back=text_list[1][0]
save_dir_back=text_list[1][1]

In [9]:
tf.reset_default_graph()
with open(os.path.join(save_dir, 'config.pkl'), 'rb') as f:
    saved_args = cPickle.load(f)
with open(os.path.join(save_dir, 'words_vocab.pkl'), 'rb') as f:
    word_keys, vocab = cPickle.load(f)

    
"""tag2words_=defaultdict(list)
for word in list(vocab.keys()):
    tag_word=nltk.pos_tag([word])
    tag2words_[tag_word[0][1]].append(word)"""

'tag2words_=defaultdict(list)\nfor word in list(vocab.keys()):\n    tag_word=nltk.pos_tag([word])\n    tag2words_[tag_word[0][1]].append(word)'

In [10]:
#FUNCTION TO SAMPLE A LINE BASED ON THEIR PROBS
#LIST MUST BE SORTED

def sampleLine(lst, cut):
    ''' samples from top "cut" lines, the distribution being the softmax of true sentence probabilities'''
    probs = list()
    for i in range(cut):
        probs.append(np.exp(lst[i][0]))
    probs = np.exp(probs) / sum(np.exp(probs))
    index = np.random.choice(cut,1,p=probs)[0]
    return lst[index][1]




In [69]:
#LOAD DICTIONARIES CREATED IN EXTRAXTING_POSTAG.IPYNB
with open('postag_dict.p','rb') as f:
    postag_dict=pickle.load(f)

#postag_dict=[postag_dict[0], tag2words_, postag_dict[2]]

In [12]:
#FUNCTION TO CREATE TEXT BETWEEN NOUN1 AND NOUN2
def recursion_postag(sess, model, vocab, prob_seq, sequence, state, temp, lists, nwords, width, first_word, word2tags, tag2words, postag_seq):
    d,s=model.compute_fx(sess, vocab, prob_seq, sequence, state, temp)
    ########
    if len(sequence)<nwords:
        ########
        postag_=postag_seq[len(sequence)]
        list_tag=list(set(tag2words[postag_]))
        ind_possible=[vocab[i] for i in list_tag]
        d_new=d[ind_possible]
        max_ind_new=d_new.argsort()[-width:][::-1]
        max_ind=[ind_possible[k] for k in max_ind_new]
        ###################
        for x in max_ind:
            new_prob=d[x]
            new_word=list(vocab.keys())[x]
            if(new_word == sequence[-1]):
                continue
            #if(partsOfSpeechFilter(sequence[-1],new_word,dictPartSpeechTags,dictPossiblePartsSpeech)):
            #    continue
            recursion_postag(sess, model, vocab, new_prob, sequence+[new_word],s, temp, lists, nwords, width, first_word, word2tags, tag2words, postag_seq)
    else:
        n_prob=d[vocab[first_word]]
        #if(partsOfSpeechFilter(sequence[-1],first_word,dictPartSpeechTags,dictPossiblePartsSpeech)):
        #    return  
        result=(0.5*(n_prob-prob_seq)+0.5*prob_seq, n_prob, sequence+[first_word], state)
        return lists.append(result)

    
#FUNCTION TO CREATE TEXT AFTER NOUN 2    
def generate_last(sess, model, vocab, prob_seq, sequence, state, temp, lists, nwords, width, first_length, tag2words, postag_seq):
    #first_length: length before the first noun
    #rest_vocab: length after second noun
    d,s=model.compute_fx(sess, vocab, prob_seq, sequence, state, temp)
    if len(sequence)<nwords-first_length:
    ########################
        postag_=postag_seq[len(sequence)+first_length]
        list_tag=list(set(tag2words[postag_]))
        ind_possible=[vocab[i] for i in list_tag]
        d_new=d[ind_possible]
        max_ind_new=d_new.argsort()[-width:][::-1]
        max_ind=[ind_possible[k] for k in max_ind_new]
        #####
        for j in max_ind:
            new_prob=d[j]
            new_word=list(vocab.keys())[j]
            if(new_word == sequence[-1]):
                continue
            #if(partsOfSpeechFilter(sequence[-1],new_word,dictPartSpeechTags,dictPossiblePartsSpeech)):
            #    continue
            generate_last(sess, model, vocab, new_prob, sequence+[new_word],s, temp, lists, nwords, width, first_length, tag2words, postag_seq)
    else:
        result=(prob_seq, sequence)
        return lists.append(result)
    
#FUNCTION TO CREATE TEXT BEFORE NOUN1
def recursion(sess, model, vocab, prob_seq, sequence, state, temp, lists, nwords, width, tag2words, postag_seq):
    d,s=model.compute_fx(sess, vocab, prob_seq, sequence, state, temp)
    if len(sequence)<nwords:
        #####
        postag_=postag_seq[-len(sequence)-1]
        list_tag=list(set(tag2words[postag_]))
        ind_possible=[vocab[i] for i in list_tag]
        d_new=d[ind_possible]
        max_ind_new=d_new.argsort()[-width:][::-1]
        max_ind=[ind_possible[k] for k in max_ind_new]
        for j in max_ind:
            new_prob=d[j]
            new_word=list(vocab.keys())[j]
            if(new_word == sequence[0]):
                continue
            #if(partsOfSpeechFilter(new_word,sequence[0],dictPartSpeechTags,dictPossiblePartsSpeech)):
            #    continue
            recursion(sess, model, vocab, new_prob, [new_word]+sequence,s, temp, lists, nwords, width, tag2words,postag_seq)
    else:
            result=(prob_seq, sequence)
            return lists.append(result)   


### create line

In [74]:
#function to get a template based on two words
def pos_synset(words, pos_dict):
    postag_nn=[]
    print(words)
    for word in words:
        pos_word=nltk.pos_tag([word])[0][1]
        postag_nn.append(pos_word)
    print (postag_nn)
    pos=str(postag_nn[0])+'-'+str(postag_nn[1])
    if pos in set(list(pos_dict.keys())):
        possible_templates=pos_dict[pos]
    else:
        pos=str(postag_nn[1])+'-'+str(postag_nn[0])
        if pos in set(list(pos_dict.keys())):
            possible_templates=pos_dict[pos]
        else:
            return None  
    template_list=list(random.choice(possible_templates))
    if postag_nn[0]==postag_nn[1]:
        template_list.append(words)
        return template_list
    else:
        ww=[words[postag_nn.index(postag_nn[0])], words[postag_nn.index(postag_nn[1])]]
        template_list.append(ww)
        return template_list
    """
    except KeyError:
        try:
            pos=str(postag_nn[1])+'-'+str(postag_nn[0])
            template_list=list(random.choice(pos_dict[pos]))
            ww=[nouns[postag_nn.index(postag_nn[1])], nouns[postag_nn.index(postag_nn[0])]]
            template_list.append(ww)
            return template_list
        except KeyError:
            print ('POS NOT IN MODEL')
            return None
            """

In [23]:
#generates one line based on two given words
def generate_line(word1, word2):
    nouns=[word1, word2]

    postag_nn=[]
    #SELECT A POS SAMPLE
    """for noun in nouns:
        postag_nn.append(nltk.pos_tag([noun])[0][1])
    if postag_nn.count('NN')>1:
        postag=list(random.choice(postag_dict[0]['NN-NN']))
        nouns=nouns
        postag.append(nouns)
    elif postag_nn.count('NN')==1 and postag_nn.count('NNS')==1:
        postag=list(random.choice(postag_dict[0]['NN-NNS']))
        nouns=[nouns[postag_nn.index('NN')], nouns[postag_nn.index('NNS')]]
        postag.append(nouns)
    elif postag_nn.count('NNS')>1:
        postag=list(random.choice(postag_dict[0]['NNS-NNS']))
        nouns=nouns
        postag.append(nouns)
    else:
        print ('WORDS ARE NOT NOUNS')
        #return None
        """
    postag=pos_synset(nouns, postag_dict[0])
    if postag==None:
        return None
    #postag=[['CC', 'PRP', 'RB', 'NN', 'IN', 'DT', 'NN', 'IN', 'NN'], [3,6], ['night', 'moon']]    

    print (postag[0])



    last_position=np.argmax(postag[1])
    first_position=np.argmin(postag[1])


    last_noun=postag[2][last_position]
    first_noun=postag[2][first_position]

    print ('First noun: '+ str(first_noun))
    print ('Last noun: '+ str(last_noun))
    first_noun=[first_noun]

    print ('#################################\n')


    #OPEN MODEL FROM LEFT TO RIGHT
    tf.reset_default_graph()
    with open(os.path.join(save_dir, 'config.pkl'), 'rb') as f:
        saved_args = cPickle.load(f)
    with open(os.path.join(save_dir, 'words_vocab.pkl'), 'rb') as f:
        word_keys, vocab = cPickle.load(f)
    model = Model_forw(saved_args, True)


    #RUN RECURSION_POSTAG BASED ON NOUN1 AND NOUN2
    tt_list=[]
    with tf.Session() as sess:
            tf.global_variables_initializer().run()
            saver = tf.train.Saver(tf.global_variables())
            ckpt = tf.train.get_checkpoint_state(save_dir)
            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(sess, ckpt.model_checkpoint_path)
                saver.restore(sess, ckpt.model_checkpoint_path)
                #score, init_state, start=init_input(sess, model, start)
                init_state = sess.run(model.initial_state)
                score = np.array([[0]])
                recursion_postag(sess, model, vocab, score, first_noun, init_state, 1, tt_list, postag[1][last_position]-postag[1][first_position], 10, last_noun,postag_dict[2], postag_dict[1], postag[0][postag[1][first_position]:postag[1][last_position]+1])
                tt_list=sorted(tt_list, key=lambda tup: tup[0], reverse=True)


    #RUN GENERATE_LAST BASED ON OUTPUTS OF RECURSION_POSTAG
    tt_list_2=[]
    with tf.Session() as sess:
            tf.global_variables_initializer().run()
            saver = tf.train.Saver(tf.global_variables())
            ckpt = tf.train.get_checkpoint_state(save_dir)

            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(sess, ckpt.model_checkpoint_path)
                saver.restore(sess, ckpt.model_checkpoint_path)
                tt_list=tt_list[:50]
                for x in tt_list:
                    generate_last(sess, model, vocab, x[1], x[2],x[3], 1, tt_list_2, len(postag[0]), 10,postag[1][first_position], postag_dict[1], postag[0])
                tt_list_2=sorted(tt_list_2, key=lambda tup: tup[0], reverse=True)

    #LOAD MODEL RIGHT TO LEFT

    tf.reset_default_graph()
    with open(os.path.join(save_dir_back, 'config.pkl'), 'rb') as f:
        saved_args_back = cPickle.load(f)
    with open(os.path.join(save_dir_back, 'words_vocab.pkl'), 'rb') as f:
        word_keys_back, vocab_back = cPickle.load(f)
    model_back = Model_back(saved_args_back, True)

    #RUN RECURSION (FIRST PART OF SENTENCE)

    tt_list_3=[]
    with tf.Session() as sess:
            tf.global_variables_initializer().run()
            saver = tf.train.Saver(tf.global_variables())
            ckpt = tf.train.get_checkpoint_state(save_dir_back)

            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(sess, ckpt.model_checkpoint_path)
                saver.restore(sess, ckpt.model_checkpoint_path)
                for x in tt_list_2[:50]:
                    init_score, init_state=model_back.score_a_list_new(sess, vocab_back, x[1])
                    recursion(sess, model_back, vocab_back, init_score, x[1], init_state, 1, tt_list_3, len(postag[0]), 8, postag_dict[1],postag[0])
                tt_list_3=sorted(tt_list_3, key=lambda tup: tup[0], reverse=True)

    #PRINT SAMPLE            
    print ('#################################\n')

    print (' '.join(sampleLine(tt_list_3, 8)))
    return tt_list_3[:15]



#create poem
def poem():
    def create_poem(seed):
        result=[]
        mp_glove=Meta_Poetry_Glove()
        five=mp_glove.five_word_algorithm_glove(seed)
        five=[tup[0] for tup in five]
        mp_glove.print_five_words_glove(seed)
        print(' ')
        if bool_five_words(five, vocab)==False:
            print ('WORDS NOT IN VOCAB, PLEASE INSERT A DIFFERENT SEED')
            return None
        else:
            list_pairs=create_pairs(five)
            for pair in list_pairs:
                line=generate_line(pair[0], pair[1])
                if line!=None:
                    result.append(line)
                else:
                    return None
            print ('*************')
            for l in result:
                print (l)
            print ('*************')
            return result
    poem=None
    #do this until all words in vocab and all pos of 5 words are on the dict
    list_vocab=list(vocab.keys())
    while poem==None:
        aux=random.sample(list_vocab, 1)
        poem=create_poem(aux[0])


In [84]:
poem()

C:\Users\hp\Documents\Data_Science\Transversal_Glove.py:37: RuntimeWarning: invalid value encountered in true_divide
  avg_w2 /= np.sqrt(np.sum(avg_w2 ** 2))


tightly->tight->squeeze~~>obtain->receive
 
WORDS NOT IN VOCAB, PLEASE INSERT A DIFFERENT SEED


UnboundLocalError: local variable 's1' referenced before assignment

In [74]:
k=generate_line('sadness', 'weather')

['NN', 'WRB', 'PRP', 'VBD', 'TO', 'DT', 'NN']
First noun: weather
Last noun: sadness
#################################

INFO:tensorflow:Restoring parameters from save_3\model.ckpt-63249
INFO:tensorflow:Restoring parameters from save_3\model.ckpt-63249
INFO:tensorflow:Restoring parameters from save_3\model.ckpt-63249
INFO:tensorflow:Restoring parameters from save_3\model.ckpt-63249
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
#################################

weather when it gave to a sadness


In [77]:
k=generate_line('head', 'hat')

['IN', 'NN', 'CC', 'NN', 'TO', 'VB']
First noun: hat
Last noun: head
#################################

INFO:tensorflow:Restoring parameters from save_3\model.ckpt-63249
INFO:tensorflow:Restoring parameters from save_3\model.ckpt-63249
INFO:tensorflow:Restoring parameters from save_3\model.ckpt-63249
INFO:tensorflow:Restoring parameters from save_3\model.ckpt-63249
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
#################################

of hat or head to say


In [79]:
print (' '.join(sampleLine(k, 15)))

on hat and head to have


In [82]:
k=generate_line('embarrassment', 'feelings')

['CC', 'VBD', 'DT', 'NNS', 'IN', 'DT', 'NN']
First noun: feelings
Last noun: embarrassment
#################################

INFO:tensorflow:Restoring parameters from save_3\model.ckpt-63249
INFO:tensorflow:Restoring parameters from save_3\model.ckpt-63249
INFO:tensorflow:Restoring parameters from save_3\model.ckpt-63249
INFO:tensorflow:Restoring parameters from save_3\model.ckpt-63249
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699


C:\Users\hp\Documents\Data_Science\model_back.py:207: RuntimeWarning: divide by zero encountered in log
  dist = p.squeeze() + np.log(probs.squeeze())


#################################

and were no feelings of all embarrassment


In [84]:
k=generate_line('property', 'land')

['DT', 'NN', 'VBD', 'RB', 'DT', 'NN', 'MD', 'VB']
First noun: land
Last noun: property
#################################

INFO:tensorflow:Restoring parameters from save_3\model.ckpt-63249
INFO:tensorflow:Restoring parameters from save_3\model.ckpt-63249
INFO:tensorflow:Restoring parameters from save_3\model.ckpt-63249
INFO:tensorflow:Restoring parameters from save_3\model.ckpt-63249
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
#################################

the land was just a property must be


In [87]:
print (' '.join(sampleLine(k, 15)))

the land was still the property must be


In [83]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

In [4]:
seed='chicken'
mp_glove=Meta_Poetry_Glove()
five=mp_glove.five_word_algorithm_glove(seed)
mp_glove.print_five_words_glove(seed)

organism->animal->food~~>irresolute->uncertain


C:\Users\hp\Documents\Data_Science\Transversal_Glove.py:37: RuntimeWarning: invalid value encountered in true_divide
  avg_w2 /= np.sqrt(np.sum(avg_w2 ** 2))
